In [28]:
import numpy as np
import matplotlib.pyplot as plt
import time
import serial
import datetime
import sys

In [29]:
# 1 rev = dist traveled is pitch of threaded rod

pitch = .8 # distance between threads in metric units (cm)
step_ang = 1.8 # degrees
steps_per_rev_x = 400 # number of steps to travel the distance equivalent to pitch
steps_per_rev_y = 400 # number of steps to travel the distance equivalent to pitch

In [75]:
# Establish a serial connection with Arduino

ser = serial.Serial('/dev/cu.usbmodem14201', 9600)

In [76]:
# Calibration: returns number of steps it takes to go from one side to the other in x and y

cal = 'Calibration' 

ser.write(str.encode(cal)) #send string to Arduino

stepsx_tot = 0
stepsy_tot = 0

count = 0
while count < 1: 
    stepsx_tot = (str(ser.readline()))
    stepsy_tot = (str(ser.readline()))
    count +=1


SerialException: read failed: [Errno 6] Device not configured

In [32]:
# prints number of steps it takes to go from one side to the other in x and y


print(stepsx_tot)
print(stepsy_tot)


b'10261\r\n'
b'394\r\n'


In [50]:
# User decides where they want the defined zero position to be from the home position using keyboard inputs to \
# move the collimator. The location from the home position is recorded and stored as variables ZeroX and ZeroY. 

ZeroX = 0
ZeroY = 0

while True:
    move = input("Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'")
    if move == 'u':
        ZeroY += pitch
        inputy=('y,' + str(steps_per_rev_y) + ',1')
        ser.write(str.encode(inputy))
    elif move == 'd':
        ZeroY -= pitch
        inputy=('y,' + str(steps_per_rev_y) + ',0')
        ser.write(str.encode(inputy))
    elif move == 'r':
        ZeroX += pitch
        inputx=('x,' + str(steps_per_rev_x) + ',0')
        ser.write(str.encode(inputx))
    elif move == 'l':
        ZeroX -= pitch
        inputx=('x,' + str(steps_per_rev_x) + ',1')
        ser.write(str.encode(inputx))
    elif move == 'done':
        break

Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'd
Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'd
Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'd
Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'd
Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'd
Input direction you wish to move to move collimator to zero position: 'u' for up, 'd' for down, 'l' for left and 'r' for right. When you are done, type 'done'd
Input direction you wish to move to move

In [7]:
#Prints location of new zero position from home position

''''
print(ZeroX)
print(ZeroY)
'''

"'\nprint(ZeroX)\nprint(ZeroY)\n"

In [8]:
# Determining region to scan from zero position

In [9]:
Length_L = float(input("Input horizontal distance from center of detector to left end of region you want to scan (in cm) less than 20 cm"))

Input horizontal distance from center of detector to left end of region you want to scan (in cm) less than 20 cm.5


In [10]:
Length_R = float(input("Input horizontal distance from center of detector to right end of region you want to scan (in cm) less than 20 cm"))

Input horizontal distance from center of detector to right end of region you want to scan (in cm) less than 20 cm1


In [13]:
# Checks to make sure inputs are valid

if (Length_L + Length_R) > 20:
    sys.exit("ERROR: Horizontal distance cannot be greater than 20 cm")
    
if (ZeroX + Length_R) > 20:
    sys.exit("ERROR: Selected scanning region is too big in the x direction. Readjust right scanning distance.")
    
if (ZeroX - Length_L) < 0:
    sys.exit("ERROR: Selected scanning region is too big in the x direction. Readjust left scanning distance.")

SystemExit: ERROR: Selected scanning region is too big in the x direction. Readjust left scanning distance.

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
Height_B = float(input("Input vertical distance from center of detector to bottom end of region you want to scan (in cm) less than 40 cm"))

Input vertical distance from center of detector to bottom end of region you want to scan (in cm) less than 40 cm1


In [15]:
Height_T = float(input("Input vertical distance from center of detector to top end of region you want to scan (in cm) less than 40 cm"))

Input vertical distance from center of detector to top end of region you want to scan (in cm) less than 40 cm1


In [16]:
# Checks to make sure inputs are valid

if (Height_B + Height_T) > 40:
    sys.exit("ERROR: Vertical distance cannot be greater than 40 cm")
    
if (ZeroY + Height_T) > 40:
    sys.exit("ERROR: Selected scanning region is too big in the y direction. Readjust top scanning distance.")
    
if (ZeroY - Height_B) < 40:
    sys.exit("ERROR: Selected scanning region is too big in the y direction. Readjust bottom scanning distance.")

SystemExit: ERROR: Selected scanning region is too big in the y direction. Readjust bottom scanning distance.

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
# position collimator to starting position in top left corner

start_x = Length_L
start_y = Height_T

In [18]:
# steps to take to travel dist to starting position

start_stepsx = (start_x/pitch) * steps_per_rev_x
start_stepsy = (start_y/pitch) * steps_per_rev_y

In [19]:
# Create a string to send to Arduino telling it which motor to move, how many steps, and in which direction

inputx=('x,' + str(start_stepsx) + ',1')
inputy=('y,' + str(start_stepsy) + ',1')

In [20]:
# Sends the string to the Arduino and motors move

ser.write(str.encode(inputx))
time.sleep(2)
ser.write(str.encode(inputy))

9

In [21]:
# granularity - distance between center of scanned spots (in cm)

gran = float(input("Input the granularity (distance between the center of scanned spots in cm)"))

Input the granularity (distance between the center of scanned spots in cm)1


In [22]:
# Check to make sure that input is valid

if gran > (Length_L + Length_R) or gran > (Height_B + Height_T):
    sys.exit("ERROR: Incompatible input. Granularity cannot be larger than scanning region")

In [23]:
# time for source to be at each position over detector

wait = float(input("Input time for source to be at each collection spot (in seconds)"))

Input time for source to be at each collection spot (in seconds)2


In [24]:
# Open a text file to store positions and corresponding time during the scan

Time_and_Positions = open(r'/Users/Elizabeth/Desktop/sourcepositions.txt', 'w+')

In [27]:
# Positioning algorithm

# Determine total x and y distance traveled. This is used to know whether the collimator has reached the end of \
# the scanning region in x and y directions
x_travel_dist = Length_L + Length_R
y_travel_dist = Height_B + Height_T

#Set starting coordinates of collimator, which will be changed as the collimator moves
x_dist = 0
y_dist = y_travel_dist

x_pos = x_dist + (ZeroX - Length_L)
y_pos = y_dist + (ZeroY - Height_B)

#Record initial starting place position and time and collect data
t=0
while t <= wait: #wait over area for specified amount of time so data can be collected
    date = datetime.datetime.now().timestamp() #current unix timestamp
    Time_and_Positions.write("%5.2f %5.2f %5.2f\n" % (date, x_dist , y_dist)) #write timestamp and x and y position to text file
    time.sleep(1)
    t += 1


while y_dist >= 0:
    if x_dist <= (x_travel_dist + gran):
        #move over right to next region to be scanned
        move_x = (gran/pitch) * steps_per_rev_x #num of steps to take to move to next scanned region
        x_dist += gran #determining x position
        x_pos = x_dist + (ZeroX - Length_L)
        inputmovex = 'x,' + str(move_x) + ',0' #string formation to send to Arduino to move x motor right to next region
        print('x =', end =" ") #print progress update
        print(x_pos)
        ser.write(str.encode(inputmovex)) #send string to Arduino to move x motor
        t=0
        while t <= wait: #wait over area for specified amount of time so data can be collected
            date = datetime.datetime.now().timestamp() #current unix timestamp
            Time_and_Positions.write("%5.2f %5.2f %5.2f\n" % (date, x_pos , y_pos)) #write timestamp and x and y position to text file
            time.sleep(1)
            t += 1
        
    else:
        if y_dist > 0:
            #move motor to end right
            inputmovex_toendR = 'xEND,' + '0' #string formation to send to Arduino to move x motor right to switch
            print('xend') #print progress update
            ser.write(str.encode(inputmovex_toendR)) #send string to Arduino
            time.sleep(5)
            #move motor to end left
            inputmovex_toendL = 'xEND,' + '1' #string formation to send to Arduino to move x motor left to switch and count steps
            print('xcount')
            ser.write(str.encode(inputmovex_toendL)) #send string to Arduino
            count=0 #wait for response from Arduino to get number of steps it took to get to the end 
            while count < 1: 
                stepsx_toend = (str(ser.readline()))
                count +=1
                
            #if stepsx_toend != stepsx_tot:
               # sys.exit("ERROR: System not calibrated. Total horizontal distnace different than calibration")
            
            time.sleep(3)
            
            #move down a row
            move_y = (gran/pitch) * steps_per_rev_y #num of steps to take to move down to next row of scanned region
            y_dist -= gran #determining y position
            x_dist = 0 #resetting x position back to 0
            x_pos = x_dist + (ZeroX - Length_L)
            y_pos = y_dist + (ZeroY - Height_B)
            inputmovey = 'y,' + str(move_y) + ',0' #string formation to send to Arduino to move y motor down
            print('y =', end =" ") #print progress update
            print(y_pos)
            ser.write(str.encode(inputmovey)) #send string to Arduino
            time.sleep(3)

            #move back to starting x position on left side
            reset = ZeroX - Length_L #distance needed to travel in x to start at beginning of scanning region from left side
            move_x = (reset/pitch) * steps_per_rev_x #num of steps to take to move right to start of scanning region in x
            inputmovex = 'x,' + str(move_x) + ',1' #string formation to send to Arduino to move x motor right
            print('xreset') #print progress update
            ser.write(str.encode(inputmovex)) #send string to Arduino
            t=0
            while t <= wait: #wait over area for specified amount of time so data can be collected
                date = datetime.datetime.now().timestamp()
                Time_and_Positions.write("%5.2f %5.2f %5.2f\n" % (date, x_pos , y_pos))
                time.sleep(1)
                t += 1
        else:
            inputmovey_toend = 'yEND,' + '0' #string formation to send to Arduino to move y motor down to switch and count steps
            print('yend') #print progress update
            ser.write(str.encode(inputmovey_toend)) #send string to Arduino
            count=0
            while count < 1: #wait for response from Arduino to get number of steps it took to get to the end
                stepsy_toend = (str(ser.readline()))
                count +=1
                
            #if stepsx_toend != stepsx_tot:
               # sys.exit("ERROR: System not calibrated. Total horizontal distnace different than calibration")  
                
            print("scan complete")
            Time_and_Positions.close() #close text file
            break 
    

x = 1.0
x = 2.0
x = 3.0
x = 4.0
xend
xcount
y = 1.0
xreset
x = 1.0
x = 2.0
x = 3.0
x = 4.0
xend
xcount
y = 0.0
xreset
x = 1.0
x = 2.0
x = 3.0
x = 4.0
yend
scan complete


In [49]:
'''
print(stepsx_toend)
print(stepsy_toend)
'''

b'42\r\n'
b'1662\r\n'


In [43]:
# reset to specified zero position

reset_x = Length_R
reset_y = Height_B

reset_stepsx = (reset_x/pitch) * steps_per_rev_x
reset_stepsy = (reset_y/pitch) * steps_per_rev_y


inputx=('x,' + str(reset_stepsx) + ',1')
inputy=('y,' + str(reset_stepsy) + ',1')

ser.write(str.encode(inputx))
time.sleep(5)
ser.write(str.encode(inputy))


"\n# reset to specified zero position\n\nreset_x = Length_R\nreset_y = Height_B\n\nreset_stepsx = (reset_x/pitch) * steps_per_rev_x\nreset_stepsy = (reset_y/pitch) * steps_per_rev_y\n\n\ninputx=('x,' + str(reset_stepsx) + ',1')\ninputy=('y,' + str(reset_stepsy) + ',1')\n\nser.write(str.encode(inputx))\ntime.sleep(5)\nser.write(str.encode(inputy))\n"